In [23]:
def sort_and_merge(intervals):
    # sort the interval list by the first element
    intervals = sorted(intervals, key=lambda x: x[0])
    merged = []
    for interval in intervals:
        # if merged is empty or non-overlapping, just add
        if not merged or merged[-1][1] < interval[0]:
            merged.append(interval)
        else: # update the last interval if overlapped
            merged[-1][1] = max(merged[-1][1], interval[1])
    #print(merged)
    return merged

In [24]:
f = sort_and_merge
assert f([[0, 3], [2, 13], [25, 28], [15, 40]]) == [[0, 13] , [15, 40]]
assert f([[1, 3], [0, 6], [11, 22], [7, 38]]) == [[0, 6] ,[7, 38]]


In [27]:
from datetime import datetime, date, timedelta
time_1 = datetime(2024, 11, 20, 9, 0, 0)
interval_1 = [time_1, time_1 + timedelta(minutes=30)]
interval_2 = [time_1 + timedelta(minutes=40), time_1 + timedelta(minutes=60)]
assert interval_1[1] < interval_2[0]

In [28]:
# generate sample records for testing
def generate_sample():
    """
    Generate sample records for testing

    Returns:
        opening: dict, dictionary containing times when the doctor is available to take patients
        appointment: dict, dictionary containing times when the doctor is not available 
    """
    opening = {}
    appointment = {}

    # generate opening sample
    morning_begin = datetime(2024, 11, 20, 9, 0, 0)
    morning_end = datetime(2024, 11, 20, 11, 30, 0)
    afternoon_begin = datetime(2024, 11, 20, 13, 0, 0)
    afternoon_end = datetime(2024, 11, 20, 16, 00, 0)
    for i in range(10):
        element = []
        element.append((morning_begin + timedelta(days=i+1), morning_end + timedelta(days=i+1)))
        element.append((afternoon_begin + timedelta(days=i+1), afternoon_end + timedelta(days=i+1)))
        opening[(morning_begin + timedelta(days=i+1)).date()] = element

    # generate appointment sample
    for i in range(10):
        element = []
        morning_begin = morning_begin + timedelta(days=1)
        afternoon_begin = afternoon_begin + timedelta(days=1)
        morning_from = morning_begin + timedelta(hours = 1)
        afternoon_from_1 = afternoon_begin + timedelta(hours = 0)
        afternoon_from_2 = afternoon_begin + timedelta(hours = 1)
        
        element.append((morning_from, morning_from + timedelta(minutes = 30)))
        element.append((afternoon_from_1, afternoon_from_1 + timedelta(minutes = 30)))
        element.append((afternoon_from_2, afternoon_from_2 + timedelta(minutes = 50)))
        appointment[(morning_begin).date()] = element
    return opening, appointment

In [30]:
opening, appointment = generate_sample()

In [33]:
# 1. sort and merge interval if overlapped by per day
merged_open = {}
merged_appoint = {}
for key, value in opening.items():
    merged_open[key] = sort_and_merge(value)

for key, value in appointment.items():
    merged_appoint[key] = sort_and_merge(value)


In [46]:
merged_open

{datetime.date(2024, 11, 21): [(datetime.datetime(2024, 11, 21, 9, 0),
   datetime.datetime(2024, 11, 21, 11, 30)),
  (datetime.datetime(2024, 11, 21, 13, 0),
   datetime.datetime(2024, 11, 21, 16, 0))],
 datetime.date(2024, 11, 22): [(datetime.datetime(2024, 11, 22, 9, 0),
   datetime.datetime(2024, 11, 22, 11, 30)),
  (datetime.datetime(2024, 11, 22, 13, 0),
   datetime.datetime(2024, 11, 22, 16, 0))],
 datetime.date(2024, 11, 23): [(datetime.datetime(2024, 11, 23, 9, 0),
   datetime.datetime(2024, 11, 23, 11, 30)),
  (datetime.datetime(2024, 11, 23, 13, 0),
   datetime.datetime(2024, 11, 23, 16, 0))],
 datetime.date(2024, 11, 24): [(datetime.datetime(2024, 11, 24, 9, 0),
   datetime.datetime(2024, 11, 24, 11, 30)),
  (datetime.datetime(2024, 11, 24, 13, 0),
   datetime.datetime(2024, 11, 24, 16, 0))],
 datetime.date(2024, 11, 25): [(datetime.datetime(2024, 11, 25, 9, 0),
   datetime.datetime(2024, 11, 25, 11, 30)),
  (datetime.datetime(2024, 11, 25, 13, 0),
   datetime.datetime(2024

In [35]:
# 2. substract the intersection of two intervals
# based on the assumption: end > start, appointment is subset of opening
def subtract_two_intervals(interval_opening, interval_appoint):
    if interval_opening[0] < interval_appoint[0]:    
       if interval_opening[1] == interval_appoint[1]:    # (3), same end, differ start
           return [[interval_opening[0], interval_appoint[0]]]
       else:
           return [[interval_opening[0], interval_appoint[0]], [interval_appoint[1], interval_opening[1]]]
    else:  # same start 
        if interval_opening[1] == interval_appoint[1]:   # (1), same end
            return [[]]
        else:
            return [[interval_appoint[1], interval_opening[1]]]  # (2), differ end
        

In [49]:
# 3. get the availability
# 2. subtract the appointment time from the merged intervals
def nobug_subtract(opening, appointment):
    """
    Subtract the appointed time from the opening time

    Args:
        opening: 2-d list: [[x, y]]
        appointment: 2-d list: [[x, y]]

    Returns:
        available: 2d list
    """
    available = []
    # sort and merge overlapped time interval
    opening = sort_and_merge(opening)
    appointment = sort_and_merge(appointment)
    #print(opening, appointment)
    # interate and update
    #myopening = iter(opening)
    opening_ind = 0
    # Done bug, since the length is not write
    appoint_ind = 0
    while appoint_ind < len(appointment):
        appoint = appointment[appoint_ind]
        if not available or (available[-1][1] < appoint[0]):
            print(opening_ind)
            available.append(opening[opening_ind])
            opening_ind += 1
        else:
            res_interval = subtract_two_intervals(available[-1], appoint)
            appoint_ind += 1
            if res_interval != [[]]:
                available = available[:-1] + res_interval
            else:
                #print("here", available[:-1])
                available = available[:-1]
    while opening_ind < len(opening):
        available.append(opening[opening_ind])
        opening_ind +=1
    return available

In [50]:
count = 0
available = {}
for date in merged_open.keys():
    if count > 1:
        break
    available[date] = nobug_subtract(merged_open[key], merged_appoint[key])
    count+=1

0
1
0
1


In [51]:
available

{datetime.date(2024, 11, 21): [[datetime.datetime(2024, 11, 30, 9, 0),
   datetime.datetime(2024, 11, 30, 10, 0)],
  [datetime.datetime(2024, 11, 30, 10, 30),
   datetime.datetime(2024, 11, 30, 11, 30)],
  [datetime.datetime(2024, 11, 30, 13, 30),
   datetime.datetime(2024, 11, 30, 14, 0)],
  [datetime.datetime(2024, 11, 30, 14, 50),
   datetime.datetime(2024, 11, 30, 16, 0)]],
 datetime.date(2024, 11, 22): [[datetime.datetime(2024, 11, 30, 9, 0),
   datetime.datetime(2024, 11, 30, 10, 0)],
  [datetime.datetime(2024, 11, 30, 10, 30),
   datetime.datetime(2024, 11, 30, 11, 30)],
  [datetime.datetime(2024, 11, 30, 13, 30),
   datetime.datetime(2024, 11, 30, 14, 0)],
  [datetime.datetime(2024, 11, 30, 14, 50),
   datetime.datetime(2024, 11, 30, 16, 0)]]}